<a href="https://colab.research.google.com/github/bcasazharuddin/Datamining/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Assignment 1 :** Take a classification problem/dataset of your choice and then compare the performance of three models namely Decision Tree  , k-NN, and Bayesian classifier over the selected task. 
     

In [53]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#label encoders
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import math
from sklearn import naive_bayes
from sklearn.preprocessing import OneHotEncoder, LabelEncoder,StandardScaler
from sklearn.tree import plot_tree


# **Load DataSet**

In [29]:
df = pd.read_csv('https://raw.githubusercontent.com/bcasazharuddin/Datamining/main/income_evaluation.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [30]:
#check out the data set
df.shape

(32561, 15)

In [31]:
#Dataset information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              32561 non-null  int64 
 1    workclass       32561 non-null  object
 2    fnlwgt          32561 non-null  int64 
 3    education       32561 non-null  object
 4    education-num   32561 non-null  int64 
 5    marital-status  32561 non-null  object
 6    occupation      32561 non-null  object
 7    relationship    32561 non-null  object
 8    race            32561 non-null  object
 9    sex             32561 non-null  object
 10   capital-gain    32561 non-null  int64 
 11   capital-loss    32561 non-null  int64 
 12   hours-per-week  32561 non-null  int64 
 13   native-country  32561 non-null  object
 14   income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [32]:
#Fixing the columns

for x in df.columns:
    x_new = x.strip()
    df=df.rename(columns={x:x_new})

df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

In [33]:
data = df.drop(["fnlwgt","capital-gain","capital-loss","native-country"],axis=1)
data

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,hours-per-week,income
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,40,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,<=50K
...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,38,<=50K
32557,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,>50K
32558,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,40,<=50K
32559,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,20,<=50K


In [34]:
#Remove Whitespace from dataframe

for column in data[["workclass","education","marital-status","occupation","race","sex"]]:
    data[column] = data[column].str.strip()

In [35]:
#categorical values of workclass
print("Unique values of workclass: ", data["workclass"].unique())
#categorical values of education
print("Unique values of education: ", data["education"].unique())
#categorical values of marital status
print("Unique values of marital status: ", data["marital-status"].unique())
#categorical values of occupation
print("Unique values of occupation: ", data["occupation"].unique())
#categorical values of relationship
print("Unique values of relationship: ", data["relationship"].unique())
#categorical values of race
print("Unique values of race: ", data["race"].unique())
#categorical values of sex
print("Unique values of sex: ", data["sex"].unique())

Unique values of workclass:  ['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov' '?'
 'Self-emp-inc' 'Without-pay' 'Never-worked']
Unique values of education:  ['Bachelors' 'HS-grad' '11th' 'Masters' '9th' 'Some-college' 'Assoc-acdm'
 'Assoc-voc' '7th-8th' 'Doctorate' 'Prof-school' '5th-6th' '10th'
 '1st-4th' 'Preschool' '12th']
Unique values of marital status:  ['Never-married' 'Married-civ-spouse' 'Divorced' 'Married-spouse-absent'
 'Separated' 'Married-AF-spouse' 'Widowed']
Unique values of occupation:  ['Adm-clerical' 'Exec-managerial' 'Handlers-cleaners' 'Prof-specialty'
 'Other-service' 'Sales' 'Craft-repair' 'Transport-moving'
 'Farming-fishing' 'Machine-op-inspct' 'Tech-support' '?'
 'Protective-serv' 'Armed-Forces' 'Priv-house-serv']
Unique values of relationship:  [' Not-in-family' ' Husband' ' Wife' ' Own-child' ' Unmarried'
 ' Other-relative']
Unique values of race:  ['White' 'Black' 'Asian-Pac-Islander' 'Amer-Indian-Eskimo' 'Other']
Unique values of sex:  [

In [36]:
#workclass labels
lb_workclass = preprocessing.LabelEncoder()
lb_workclass.fit(["Private","Self-emp-not-inc","Local-gov","?",
                  "State-gov","Self-emp-inc",
                 "Federal-gov","Without-pay","Never-worked"])
data.iloc[:,1] = lb_workclass.transform(data.iloc[:,1])

#education labels
lb_educ = preprocessing.LabelEncoder()
lb_educ.fit(["HS-grad","Some-college","Bachelors","Masters",
             "Assoc-voc","11th","Assoc-acdm","10th","7th-8th","Prof-school",
             "9th","12th","Doctorate","5th-6th","1st-4th","Preschool"])
data.iloc[:,2] = lb_educ.transform(data.iloc[:,2])

#marriage labels
lb_marry = preprocessing.LabelEncoder()
lb_marry.fit(["Married-civ-spouse","Never-married","Divorced","Separated",
              "Widowed","Married-spouse-absent","Married-AF-spouse"])
data.iloc[:,4] = lb_marry.transform(data.iloc[:,4])

#occupation labels
lb_occ = preprocessing.LabelEncoder()
lb_occ.fit(['Adm-clerical', 'Exec-managerial', 'Handlers-cleaners',
       'Prof-specialty', 'Other-service', 'Sales', 'Craft-repair',
       'Transport-moving', 'Farming-fishing', 'Machine-op-inspct',
       'Tech-support', '?', 'Protective-serv', 'Armed-Forces',
       'Priv-house-serv'])
data.iloc[:,5] = lb_occ.transform(data.iloc[:,5])

#relationship labels
lb_rel = preprocessing.LabelEncoder()
lb_rel.fit([' Not-in-family', ' Husband', ' Wife', ' Own-child', ' Unmarried',
       ' Other-relative'])
data.iloc[:,6] = lb_rel.transform(data.iloc[:,6])

#race labels
lb_race = preprocessing.LabelEncoder()
lb_race.fit(['White', 'Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo',
       'Other'])
data.iloc[:,7] = lb_race.transform(data.iloc[:,7])

#gender labels
lb_sex = preprocessing.LabelEncoder()
lb_sex.fit(['Male', 'Female'])
data.iloc[:,8] = lb_sex.transform(data.iloc[:,8])

In [37]:
data.head(8)

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,hours-per-week,income
0,39,7,9,13,4,1,1,4,1,40,<=50K
1,50,6,9,13,2,4,0,4,1,13,<=50K
2,38,4,11,9,0,6,1,4,1,40,<=50K
3,53,4,1,7,2,6,0,2,1,40,<=50K
4,28,4,9,13,2,10,5,2,0,40,<=50K
5,37,4,12,14,2,4,5,4,0,40,<=50K
6,49,4,6,5,3,8,1,2,0,16,<=50K
7,52,6,11,9,2,4,0,4,1,45,>50K


In [38]:
X=data.iloc[:,:-1]
y=data[["income"]]

# Train/Test split

In [39]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=4)

print("Size of train set:", X_train.shape, y_train.shape)
print("Size of test set: ", X_test.shape, y_test.shape)

Size of train set: (26048, 10) (26048, 1)
Size of test set:  (6513, 10) (6513, 1)


# Decision Tree

In [40]:
incomeGuess = DecisionTreeClassifier(criterion="entropy",max_depth=4)
incomeGuess

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [41]:
incomeGuess.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [42]:
predict_income = incomeGuess.predict(X_test)

In [43]:
## Evaluation
print("Accuracy of decision tree model regarding to income prediction: ", metrics.accuracy_score(y_test,predict_income))

Accuracy of decision tree model regarding to income prediction:  0.8249654537079687


# KNN

In [48]:
for i in range(1,11):
    #Train the model
    neigh=KNeighborsClassifier(n_neighbors=i).fit(X_train,np.ravel(y_train))
    y_pred=neigh.predict(X_test)
    acc=metrics.accuracy_score(y_test,y_pred)
    print(i,acc)

1 0.7707661599877169
2 0.7947182557961001
3 0.7907262398280362
4 0.8051589129433441
5 0.7970213419315215
6 0.8080761553815446
7 0.7962536465530478
8 0.8046982957162598
9 0.8031629049593122
10 0.8051589129433441


In [49]:
log_r = LogisticRegression(C=0.01,solver="liblinear").fit(X_train,np.ravel(y_train))

In [50]:
y_hat = log_r.predict(X_test)
y_hat

array([' <=50K', ' <=50K', ' >50K', ..., ' <=50K', ' <=50K', ' <=50K'],
      dtype=object)

In [51]:
con_mat = confusion_matrix(y_test,y_hat)
total_accuracy = (con_mat[0, 0] + con_mat[1, 1]) / float(np.sum(con_mat))
print(con_mat)
print('Total accuracy of income model: %.2f' % total_accuracy)

[[4677  295]
 [ 943  598]]
Total accuracy of income model: 0.81


# Naive Bayes Classification

In [54]:
clf = DecisionTreeClassifier(max_depth=5).fit(X_train, y_train)
print('Training Score: {:.2f}'.format(clf.score(X_train, y_train)))
print('Testing Score: {:.2f}'.format(clf.score(X_test, y_test)))

scaler = StandardScaler()
scaler.fit(X_train)
x_test_scaled = scaler.transform(X_test)
y_pred_n  = clf.predict(x_test_scaled)
nb = naive_bayes.GaussianNB()
nb.fit(X_train, y_train)
nb_acc = 100*nb.score(X_test, y_test)
print('Naive Bayes Predictions: \n', nb.predict(X_test), '\n Accuracy:', nb_acc, '%')


Training Score: 0.82
Testing Score: 0.83
Naive Bayes Predictions: 
 [' <=50K' ' >50K' ' >50K' ... ' <=50K' ' <=50K' ' <=50K'] 
 Accuracy: 77.09196990634116 %


/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
